In [13]:
# DIPY Experiments - m2g
# W. Gray Roncal - January 22, 2016
# Snowzilla

# Params

dwifile = 'KKI2009_113_1_DTI.nii'
fbval = 'KKI2009_113_1_DTI.bval'
fbvec = 'KKI2009_113_1_DTI.bvec'
fatlas = 'MNI152_T1_1mm_brain.nii.gz'
fatlas_labels = 'desikan.nii.gz'
import os.path
subLabel = os.path.splitext(dwifile)[0]
print subLabel

#TODO GK: build argparser, add m2rage, rename sublabel, output graphs

#create: run_m2g_dipy.py that takes in file names and prints them

KKI2009_113_1_DTI


In [14]:
# Load Data
from datetime import datetime
startTime = datetime.now()


from dipy.io import read_bvals_bvecs, read_bvec_file
from dipy.core.gradients import gradient_table
import numpy as np
import nibabel as nib

img = nib.load(dwifile)

atlas_img = nib.load(fatlas)

data = img.get_data()
atlas = atlas_img.get_data()

bvals, bvecs = read_bvals_bvecs(fbval, fbvec)

# Get rid of spurrious scans
idx = np.where((bvecs[:, 0] == 100) & (bvecs[:, 1] == 100) & (bvecs[:, 2] == 100))
bvecs = np.delete(bvecs, idx, axis=0)
bvals = np.delete(bvals, idx, axis=0)
data = np.delete(data,idx,axis=3)

gtab = gradient_table(bvals, bvecs, atol = 0.01) 

print gtab.info

B-values shape (33,)
         min 0.000000 
         max 700.000000 
B-vectors shape (33, 3)
         min -0.996763 
         max 1.000000 
None


In [15]:
# Preprocess DTI

# TODO - eddy correction!

# Get b0
b0 = np.where(gtab.b0s_mask)[0]
b0_vol = np.squeeze(data[:, :, :, b0]) #if more than 1, just grab 1 for now.
print datetime.now() - startTime

0:00:09.761512


In [16]:
# Register to atlas 
# Register DTI to atlas 
#(B0 to MNI directly)

from dipy.viz import regtools
from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.data.fetcher import fetch_syn_data, read_syn_data
from dipy.align.imaffine import (transform_centers_of_mass,
                                 AffineMap,
                                 MutualInformationMetric,
                                 AffineRegistration)
from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)

static_grid2world = atlas_img.get_affine()
moving_grid2world = img.get_affine()

# for compatibility with example code
static = atlas
moving = b0_vol

print moving.shape
print static.shape
"""
We can see that the images are far from aligned by drawing one on top of
the other. The images don't even have the same number of voxels, so in order
to draw one on top of the other we need to resample the moving image on a grid
of the same dimensions as the static image, we can do this by "transforming"
the moving image using an identity transform
"""

identity = np.eye(4)
affine_map = AffineMap(identity,
                       static.shape, static_grid2world,
                       moving.shape, moving_grid2world)
resampled = affine_map.transform(moving)


"""
We can obtain a very rough (and fast) registration by just aligning the centers
of mass of the two images
"""

c_of_mass = transform_centers_of_mass(static, static_grid2world,
                                      moving, moving_grid2world)

"""
We can now transform the moving image and draw it on top of the static image,
registration is not likely to be good, but at least they will occupy roughly
the same space
"""

transformed = c_of_mass.transform(moving)

"""
This was just a translation of the moving image towards the static image, now
we will refine it by looking for an affine transform. We first create the
similarity metric (Mutual Information) to be used. We need to specify the
number of bins to be used to discretize the joint and marginal probability
distribution functions (PDF), a typical value is 32. We also need to specify
the percentage (an integer in (0, 100]) of voxels to be used for computing the
PDFs, the most accurate registration will be obtained by using all voxels, but
it is also the most time-consuming choice. We specify full sampling by passing
None instead of an integer
"""

nbins = 32
sampling_prop = None
metric = MutualInformationMetric(nbins, sampling_prop)

"""
To avoid getting stuck at local optima, and to accelerate convergence, we use a
multi-resolution strategy (similar to ANTS [Avants11]_) by building a Gaussian
Pyramid. To have as much flexibility as possible, the user can specify how this
Gaussian Pyramid is built. First of all, we need to specify how many
resolutions we want to use. This is indirectly specified by just providing a
list of the number of iterations we want to perform at each resolution. Here we
will just specify 3 resolutions and a large number of iterations, 10000 at the
coarsest resolution, 1000 at the medium resolution and 100 at the finest. These
are the default settings
"""

level_iters = [10000, 1000, 100]

"""
To compute the Gaussian pyramid, the original image is first smoothed at each
level of the pyramid using a Gaussian kernel with the requested sigma. A good
initial choice is [3.0, 1.0, 0.0], this is the default
"""

sigmas = [3.0, 1.0, 0.0]

"""
Now we specify the sub-sampling factors. A good configuration is [4, 2, 1],
which means that, if the original image shape was (nx, ny, nz) voxels, then the
shape of the coarsest image will be about (nx//4, ny//4, nz//4), the shape in
the middle resolution will be about (nx//2, ny//2, nz//2) and the image at the
finest scale has the same size as the original image. This set of factors is
the default
"""

factors = [4, 2, 1]

"""
Now we go ahead and instantiate the registration class with the configuration
we just prepared
"""

affreg = AffineRegistration(metric=metric,
                            level_iters=level_iters,
                            sigmas=sigmas,
                            factors=factors)

"""
Using AffineRegistration we can register our images in as many stages as we
want, providing previous results as initialization for the next (the same logic
as in ANTS). The reason why it is useful is that registration is a non-convex
optimization problem (it may have more than one local optima), which means that
it is very important to initialize as close to the solution as possible. For
example, lets start with our (previously computed) rough transformation
aligning the centers of mass of our images, and then refine it in three stages.
First look for an optimal translation. The dictionary regtransforms contains
all available transforms, we obtain one of them by providing its name and the
dimension (either 2 or 3) of the image we are working with (since we are
aligning volumes, the dimension is 3)
"""

transform = TranslationTransform3D()
params0 = None
starting_affine = c_of_mass.affine
translation = affreg.optimize(static, moving, transform, params0,
                              static_grid2world, moving_grid2world,
                              starting_affine=starting_affine)

"""
If we look at the result, we can see that this translation is much better than
simply aligning the centers of mass
"""

transformed = translation.transform(moving)

"""
Now lets refine with a rigid transform (this may even modify our previously
found optimal translation)
"""

transform = RigidTransform3D()
params0 = None
starting_affine = translation.affine
rigid = affreg.optimize(static, moving, transform, params0,
                        static_grid2world, moving_grid2world,
                        starting_affine=starting_affine)

"""
This produces a slight rotation, and the images are now better aligned
"""

transformed = rigid.transform(moving)

"""
Finally, lets refine with a full affine transform (translation, rotation, scale
and shear), it is safer to fit more degrees of freedom now, since we must be
very close to the optimal transform
"""

transform = AffineTransform3D()
params0 = None
starting_affine = rigid.affine
affine = affreg.optimize(static, moving, transform, params0,
                         static_grid2world, moving_grid2world,
                         starting_affine=starting_affine)

transformed = affine.transform(moving)

print datetime.now() - startTime

(256, 256, 65)
(182, 218, 182)
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
0:18:54.496873


In [17]:
# Apply transform
# Loop through each of the DTI volumes for each of the transforms
import igraph
dwi_reg = np.zeros([transformed.shape[0],transformed.shape[1],transformed.shape[2],33])#data.shape[3]-1]) #TODO data.shape
print dwi_reg.shape

for x in range(dwi_reg.shape[3]):
    moving = np.squeeze(data[:, :, :, x])    
    transformed = c_of_mass.transform(moving)
    transformed = translation.transform(moving)
    transformed = rigid.transform(moving)
    transformed = affine.transform(moving)
    dwi_reg[:,:,:,x] = transformed
print datetime.now() - startTime


(182, 218, 182, 33)
0:20:49.714946


In [18]:
# Tensor estimation

import numpy as np

from dipy.reconst.dti import TensorModel, fractional_anisotropy
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.direction import peaks_from_model
from dipy.tracking.eudx import EuDX
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import fvtk
from dipy.viz.colormap import line_colors


data = dwi_reg

labeldata = nib.load(fatlas_labels)

label = labeldata.get_data()

"""
Create a brain mask. Here we just threshold labels.
"""

mask = (label > 0)

gtab.info
print data.shape
"""
For the constrained spherical deconvolution we need to estimate the response
function (see :ref:`example_reconst_csd`) and create a model.
"""

response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)

csd_model = ConstrainedSphericalDeconvModel(gtab, response)

"""
Next, we use ``peaks_from_model`` to fit the data and calculated the fiber
directions in all voxels.
"""

sphere = get_sphere('symmetric724')

csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             mask=mask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)

"""
For the tracking part, we will use the fiber directions from the ``csd_model``
but stop tracking in areas where fractional anisotropy (FA) is low (< 0.1).
To derive the FA, used here as a stopping criterion, we would need to fit a
tensor model first. Here, we fit the Tensor using weighted least squares (WLS).
"""
print 'tensors...'

tensor_model = TensorModel(gtab, fit_method='WLS')
tensor_fit = tensor_model.fit(data, mask)

FA = fractional_anisotropy(tensor_fit.evals)

"""
In order for the stopping values to be used with our tracking algorithm we need
to have the same dimensions as the ``csd_peaks.peak_values``. For this reason,
we can assign the same FA value to every peak direction in the same voxel in
the following way.
"""

stopping_values = np.zeros(csd_peaks.peak_values.shape)
stopping_values[:] = FA[..., None]
print datetime.now() - startTime

B-values shape (33,)
         min 0.000000 
         max 700.000000 
B-vectors shape (33, 3)
         min -0.996763 
         max 1.000000 
(182, 218, 182, 33)
tensors...
0:40:19.875360


/usr/local/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:576: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/usr/local/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:576: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/usr/local/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:576: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/usr/local/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:576: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)


In [11]:
# Fiber Tracking
# TODO: better seeding
"""
``EuDX`` [Garyfallidis12]_ is a fast algorithm that we use here to generate
streamlines. If the parameter ``seeds`` is a positive integer it will generate
that number of randomly placed seeds everywhere in the volume. Alternatively,
you can specify the exact seed points using an array (N, 3) where N is the
number of seed points. For simplicity, here we will use the first option
(random seeds). ``a_low`` is the threshold of the fist parameter
(``stopping_values``) which means that there will that tracking will stop in
regions with FA < 0.1.
"""

streamline_generator = EuDX(stopping_values,
                            csd_peaks.peak_indices,
                            seeds=10**6,
                            odf_vertices=sphere.vertices,
                            a_low=0.1)

streamlines = [streamline for streamline in streamline_generator]

print datetime.now() - startTime

0:45:22.423110


In [19]:
# Graph gen (non-scalable for now)
#print label(ss)
# initialize graph
# For each streamline, round values
# index into array and get all unique labels
# for every n choose 2 ids, increment graph
# plot graph
# save graph

startTime = datetime.now()

from itertools import combinations
import networkx as nx
import matplotlib.pylab as plt
#G = nx.Graph()


G = np.zeros((70,70))

print np.shape(streamlines)

for y in range(np.shape(streamlines)[0]):
    
    if (y % 25000) == 0:
        print y
    ss = (np.round(streamlines[y]))
    ss = ss.astype(int)
    f = []

    for x in range(ss.shape[0]):
        f.append(label[ss[x][0],ss[x][1],ss[x][2]])

    f = np.unique(f)
    f = f[f != 0]
    ff = list(combinations(f,2))

    for z in range(np.shape(ff)[0]):
        G[ff[z][0]-1,ff[z][1]-1] = G[ff[z][0]-1,ff[z][1]-1] + 1
        
print datetime.now() - startTime

(670701,)
0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000
375000
400000
425000
450000
475000
500000
525000
550000
575000
600000
625000
650000
0:02:01.974643


In [20]:
print G
import numpy as np
np.save('dipygraph.npy', G) #because things kept dying I wanted to keep the graph

[[  0.00000000e+00   8.19000000e+03   3.86100000e+03 ...,   1.22000000e+02
    4.00000000e+00   4.70000000e+01]
 [  0.00000000e+00   0.00000000e+00   2.35000000e+02 ...,   0.00000000e+00
    0.00000000e+00   4.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.30000000e+01
    0.00000000e+00   6.00000000e+00]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   3.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [22]:
import igraph as ig
g = ig.Graph.Weighted_Adjacency(list(G), mode='undirected', attr='weight')
g.save('thefilenameyouwant.graphml', format='graphml')

In [2]:


fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
G2 = G + np.transpose(G)
plt.imshow(np.log10(G2), interpolation='nearest', cmap=plt.cm.hot)
plt.colorbar()
plt.savefig(subLabel+'.png')

#plt.show()


NameError: name 'plt' is not defined

In [12]:
import csv
with open(subLabel+'.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    #for x in range(G2.shape[0]):    
    csvwriter.writerows(G2)   
    
    #TODO GK: output G as graphml

In [1]:
print G
len(G)



NameError: name 'G' is not defined

In [13]:
#plt.show()
